## Assignment 02 – Cryptocurrency Portfolio

Group 3: Shufeng Li, Wenlin Ye, Yifei Yin  
Github link: https: // github.com/yyin422/fall23-5700-group-3/tree/main/HW2


#### 1 Importing data and packages


In [2]:
import pandas_datareader.data as pdr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import requests

#### 2 Gather Coin token ID information from Coinmarket, Binance, or Coinbase.
You will need to search on any of the three websites to get the token IDs. The video shows how to do it through Binance and coin market cap. A slight difference between the Binance token and the coin market token is that the Binance token adds BUSD or USDT at the end, while the CoinMarket token does not. For example, Bitcoin is represented as BTC in CoinMarket and BTCBUSD or BTCUSDT in Binance. This represents the currency conversion to a stablecoin instead of a fiat currency.

For the sake of this bit, you can choose any 30 cryptocurrencies except Bitcoin and Ethereum. Here are the categories to select from

High market cap group - this will include the most widely traded coins. We will use the top 5 coins, namely, Bitcoin (BTC-USD), Ethereum (ETH-USD), Litecoin (LTC-USD), Teather (USDT), and BNB (BNB-USD).
Mid-market cap group will include coins with a market cap between 1 billion and 20 billion. We will use the top 5 coins, namely, Cardano (ADA-USD), Polygon (MATIC-USD), Dogecoin (DOGE-USD), Ripple (XRP-USD), and Binance USD (BUSD).
The low market cap group will include coins with a market cap between 100 million and 1 billion. We will use the top 20 coins.
Make sure to list the ticker ids in a list of strings

#### 2.1 Getting Data From Binance
Create a function that takes ticker interval, start_time, and end_time as input. We want to get all the data for the selected tickers and save them in data chosen frames. There is a way to automate this by using a function as well.

For each coin/token, you will get a dataframe of daily movements from start to end. The date and time are set as the index. Close, Symbol, and index are the most critical columns, and we want to subset them in a" coins" dataframe for all the coins.
Rename close to "Coinname_closeprice." Replace the coin name with the name of the coin and token id.

#### 2.2 Combining the dataframe
The updated dataframe should have the index and close price of all the coins of interest. We will use this to calculate the efficient frontier for the portfolio.

#### 3. Calculating the Efficient Frontier
To calculate an efficient frontier, we will need to calculate each coin's means, standard deviations, and covariances.

Calculate the mean of the dataframe generated in 2.2 using pd.df.mean() method
Calculate the covariance matrix of the dataframe generated in 2.2 using pd.df.cov() method
Use the function in the video to generate a complete crypto portfolio for $10,000.

#### 4. Locate the Best portfolios
The percentages returned from the best portfolio search represent the total assets added to each coin from the $10, 000.

#### 4.1 Locate the position of the portfolio with the highest Sharpe Ratio

#### 4.2 Locate the portfolio with the lowest volatility


#### 4.3 Plot the efficient portfolio